In [ ]:
import requests
import json

# ==========================
# Konfiguration
# ==========================

API_KEY = "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"  
MODEL = "openai/gpt-4.1-mini"

INPUT_TRANSCRIPT = "transkript_mit_sprechern.txt"
OUTPUT_TRANSCRIPT = "generiertes_transkript.txt"

# Hier kannst du das gewünschte Szenario/Thema vorgeben:
SCENARIO_TOPIC = (
    "das trinkwasser einer grossstadt soll vergiftet werden"
)
# Beispiel: "Cyberangriff auf das Leit- und Sicherungssystem eines Bahnnetzes"
# oder einfach leer lassen "" für generische Szenarien

# ==========================
# Prompt-Bausteine
# ==========================

RULES_TEXT = """
Op4C Spielregel

Allgemein
Hierbei handelt es sich um eine vereinfachte Form der Regeln des SME Matrix Wargames: Optimize for Chaos [Op4C]. In dem Spiel treten zwei Teams gegeneinander an: Rot und Blau. Rot spielt einen staatlichen Akteur, der sich zum Ziel gesetzt hat, etwas zu sabotieren, erobern oder zerstoeren. Blau spielt den Verteidiger der mit allen zur Verfügung stehenden Mitteln versucht, den Angriff abzuwehren.

Zentrales Element der Methode ist das Diskussionsgefecht: Hier treten die zwei Teams in eine spielerisch geleitete Expertendiskussion ein, wo beide Teams Argumente für bzw. gegen die Machbarkeit vom Rot geplanten Angriff austauschen. Das Spiel wird von einem Spielleiter moderiert.

Das Spiel besteht aus 2 Runden.
Jede Runde hat 2 Phasen: Intelphase und Diskussionsgefecht.
"""

INTELPHASE_TEXT = """
Phase 1: Intelphase
In der Intelphase kann Rot einen Vorteil erlangen, der sich in der nächsten Phase, dem Diskussionsgefecht, positiv auf den zu würfelnden Erfolgswurf auswirkt. Ob der Vorteil erlangt wird und wie hoch dieser ausfällt, ist vom Zufall abhängig.

Die Phase beginnt mit Blau: Blau würfelt mit 3 sechsseitigen Würfeln und versucht, mindestens eine 12 zu erreichen. Wenn das passiert, dürfen sie den Intelvorteil von Rot kennen. Wenn Blau die 12 nicht erreicht, ist der Vorteil geheim und Team Blau verlässt den Raum, bevor Rot würfelt.

Rot würfelt dann im Anschluss, welchen Intelvorteil es bekommt. Hierfür wird zwei Mal nacheinander mit einem sechsseitigen Würfel gewürfelt. Der erste Wurf bestimmt die X-Achse, der zweite die Y-Achse.

Auf der X-Achse wird bestimmt, wie hoch der Vorteil ist, den Rot erhält:
- 1–2: kein Vorteil
- 3–4: +1 auf den Erfolgswurf
- 5–6: +2 auf den Erfolgswurf

Auf der Y-Achse wird bestimmt, für welche Angriffsart der Vorteil gilt:
- 1–2: Cyberangriff
- 3–4: Angriff gegen Personal
- 5–6: Sabotageangriff

Zusammengefasst:

             | Cyber (1–2)         | Personal (3–4)         | Sabotage (5–6)
1–2 (X)      | Kein Vorteil        | Kein Vorteil           | Kein Vorteil
3–4 (X)      | +1 auf Cyber        | +1 auf Personal        | +1 auf Sabotage
5–6 (X)      | +2 auf Cyber        | +2 auf Personal        | +2 auf Sabotage
"""

DISCUSSION_TEXT = """
Phase 2: Diskussionsgefecht
Sobald die Intelphase beendet ist, kommen beide Teams wieder in den gleichen Raum und Rot stellt den Angriff vor, den es geplant hat. Hierbei spezifiziert Rot Angriffsziel, Angriffsmethode und erwartete Auswirkungen.

Die Diskussion wird entlang von 4 Dimensionen geführt:
- Ressourcen
- Angriffskomplexität
- Verteidigung
- Aktionszeitfenster

Blau versucht in jeder Dimension Argumente zu formulieren, die gegen den Angriff sprechen bzw. die Erfolgswahrscheinlichkeit reduzieren, während Rot das Gegenteil versucht. Jede Dimension wird abhängig von der Qualität der Argumente zu Gunsten von Blau oder Rot bewertet und in T-Shirt-Größen (S, M, L, XL) festgehalten (S=1, M=2, L=3, XL=4).

Wenn alle 4 Dimensionen bewertet wurden, muss Rot mindestens die Summe dieser Werte mit 3 sechsseitigen Würfeln würfeln. Gute Argumente von Rot erhöhen die Erfolgswahrscheinlichkeit, gute Argumente von Blau reduzieren sie.

Die Dimensionen im Detail:
- Ressourcen: Kosten des Angriffs relativ zu den verfügbaren Ressourcen von Rot (S = minimal, XL = alle verfügbaren)
- Verteidigung: Eignung und Erprobung bestehender Abwehrmaßnahmen des Verteidigers gegen den spezifischen Angriff (S = nicht existent, XL = vorhanden, eingeübt und perfekt zugeschnitten)
- Angriffskomplexität: Koordinationsaufwand und erforderliche Spezialkompetenz (S = nicht komplex, keine besonderen Skills nötig; XL = hochgradig komplex)
- Aktionszeitfenster: Zeit zwischen Angriffsinitiierung und unvermeidbarem Schaden (S = Stunden, M = Tage, L = Wochen–Monate, XL = Jahre)

Am Ende der Diskussionsphase werden gemeinsam die Auswirkungen des Angriffs besprochen.
Dann beginnt Runde 2 wieder mit Phase 1 (Intelphase).
"""

END_TEXT = """
Spielende
Wenn beide Runden durchgespielt wurden, wird in einer gemeinsamen Diskussion mit Rot und Blau geprüft, ob Rot erfolgreich war. Das Ergebnis wird in einem One-Pager festgehalten und an weitere Prozesse zur Maßnahmenentwicklung übergeben.
"""

TASKS_FOR_KI = """
Aufgaben an die KI:
1. Gespräch strukturiert protokollieren.
2. Sprecher unterscheiden.
3. Spielrunden und -phasen unterscheiden.
4. Ergebnisse der Intelphasen korrekt dokumentieren.
5. Ergebnisse aller Dimensionen der Phase 2 korrekt dokumentieren, inklusive Erfolgswurf von Rot.
6. Eine Ergebniszusammenfassung (One-Pager) generieren mit:
   - Teilnehmern,
   - Angriffsergebnis,
   - Verteidigungsmaßnahmen des Verteidigers,
   - Angriffsverlauf,
   - Schwachstellen, die den Angriff ermöglicht haben.
"""

GENERATION_INSTRUCTIONS = """
Du bekommst ein Beispiel-Transkript einer gespielten Wargaming- oder Planspielrunde.

Analysiere:
- Stil,
- Struktur,
- Sprecherwechsel,
- Detailgrad.

Erzeuge anschließend ein komplett neues, fiktives Transkript einer anderen Runde, basierend auf den oben beschriebenen Spielregeln. Es muss nicht 1:1 dem Beispiel entsprechen; die Runde darf inhaltlich abweichen, entscheidend ist, dass die Spielregeln eingehalten werden (Runden, Phasen, Intelwürfe, Diskussionsgefecht, Dimensionen, Erfolgswurf etc.).

Rahmenbedingungen für das neue Transkript:
- Thema, Ziele, konkrete Angriffe und Verteidigungsmaßnahmen sollen neu und fiktiv sein.
- Übernimm nur Stil und Struktur, NICHT den Inhalt des Beispiels.
- Nimm auch gerne militrische Angriffe, die dann diskutiert werden und anhand der 4 Dimensionen eingestuft werden.

Formatvorgaben:
- Verwende Sprecherlabels wie „Sprecher 1:“, „Sprecher 2:“ etc.
- Jeder Sprecherwechsel steht in einer neuen Zeile.
- Halte ungefähr die gleiche Länge und Detailtiefe wie das Beispieltranskript.
- Schreibe NUR das neue Transkript, ohne zusätzliche Erklärungen.
- Baue am Ende eine klare, strukturierte Ergebniszusammenfassung (One-Pager) gemäß den Aufgaben an die KI ein.
"""

# ==========================
# Transkript einlesen
# ==========================

with open(INPUT_TRANSCRIPT, "r", encoding="utf-8") as f:
    example_transcript = f.read()

# ==========================
# System- & User-Prompt bauen
# ==========================

# Optionaler Zusatztext für das gewünschte Szenario
if SCENARIO_TOPIC.strip():
    scenario_instruction = (
        "\n\n=== THEMATISCHER RAHMEN FÜR DIE NEUE RUNDE ===\n"
        "Das neue, fiktive Szenario soll sich inhaltlich an folgendem Thema orientieren:\n"
        f"\"\"\"{SCENARIO_TOPIC}\"\"\"\n"
        "Passe Ziele, Angriffsmethoden, betroffene Infrastruktur und Verteidigungsmaßnahmen an dieses Thema an."
    )
else:
    scenario_instruction = ""

system_text = (
    "Du bist eine KI, die Wargaming- und Planspielrunden nach vorgegebenen Regeln "
    "strukturiert protokolliert und realistische Beispielrunden generiert.\n\n"
    "Zuerst folgen die Spielregeln und der Rahmen des Op4C-Wargames.\n\n"
    "=== SPIELREGELN & RAHMEN ===\n"
    f"{RULES_TEXT}\n"
    f"{INTELPHASE_TEXT}\n"
    f"{DISCUSSION_TEXT}\n"
    f"{END_TEXT}\n\n"
    "=== AUFGABEN DER KI ===\n"
    f"{TASKS_FOR_KI}\n\n"
    "=== ANWEISUNGEN FÜR DIE NEUE GENERIERTE RUNDE ===\n"
    f"{GENERATION_INSTRUCTIONS}"
    f"{scenario_instruction}"
)

user_text = (
    "Hier ist das Beispiel-Transkript einer echten Runde:\n\n"
    "----- BEGINN BEISPIEL -----\n"
    f"{example_transcript}\n"
    "----- ENDE BEISPIEL -----\n\n"
    "Erzeuge nun eine neue, fiktive Runde entsprechend der oben beschriebenen Regeln "
    "und Anweisungen. Nutze das Beispiel nur als stilistische Vorlage."
)

payload = {
    "model": MODEL,
    "messages": [
        {"role": "system", "content": system_text},
        {"role": "user", "content": user_text},
    ],
    "stream": False,
}

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json",
    "HTTP-Referer": "https://example.com",    # optional
    "X-Title": "Wargaming-Transcript-Generator",
}

# ==========================
# Anfrage an OpenRouter senden
# ==========================

response = requests.post(
    "https://openrouter.ai/api/v1/chat/completions",
    headers=headers,
    data=json.dumps(payload),
)
response.raise_for_status()
data = response.json()

new_transcript = data["choices"][0]["message"]["content"]

# ==========================
# Neues KI-Transkript speichern
# ==========================

with open(OUTPUT_TRANSCRIPT, "w", encoding="utf-8") as f:
    f.write(new_transcript)

print("Neue Beispielrunde gespeichert in:")
print(OUTPUT_TRANSCRIPT)
print("\nVorschau:\n")
print(new_transcript[:1000])


Neue Beispielrunde gespeichert in:
generiertes_transkript.txt

Vorschau:

Sprecher 1: Guten Morgen zusammen, heute spielen wir die Op4C-Runde zum Szenario „Vergiftung der Trinkwasserversorgung in einer Großstadt“. Rot übernimmt den Angriff, Blau die Verteidigung. Wie immer kenne ich die Regeln, falls Fragen aufkommen. Rot, möchtest du dich vorstellen?

Sprecher 2: Ja, gerne. Ich bin Lisa aus Team Rot. Unser Ziel ist es, durch eine Sabotageaktion im Wasserwerk die Trinkwasserversorgung einer Großstadt zu beeinträchtigen. Der Angriff soll Verwirrung und Gesundheitsgefahren auslösen.

Sprecher 1: Schön, Lisa. Und du, Tom, bist Team Blau?

Sprecher 3: Genau. Ich bin Tom und werde versuchen, die Verteidigung möglichst effektiv zu organisieren, damit der Angriff scheitert.

Sprecher 1: Super. Dann starten wir mit Runde 1, Phase 1: Die Intelphase. Blau würfelt zuerst, um zu prüfen, ob ihr den Vorteil von Rot erkennt. Tom, bitte drei Würfel.

Sprecher 3: Ich habe eine 9 gewürfelt.

Sprecher 1: